In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

%matplotlib widget

## Memeriksa ketersedian CUDA
CUDA memanfaatkan komputasi pemrogramam paralel dengan menggunakan GPU

In [9]:
if torch.cuda.is_available():
    dev = 'cuda'
else:
    dev = 'cpu'

device = torch.device(dev)
print(f'Komputasi menggunakan {dev}...') 

Komputasi menggunakan cuda...


## Memuat Data
Memuat data, kemudian dibagi menjadi 3 bagian training set, validation set, test set.

In [10]:
# Memuat data Iris.csv
def get_data(path):
    path = '../machinel/data/Iris.csv'

    df = pd.read_csv(path)
    X = df.iloc[:,1:5].values
    X = torch.tensor(X, dtype=torch.float32)

    y = df.Species
    enc = LabelEncoder()
    y = enc.fit_transform(y)
    y = torch.from_numpy(y).type(torch.LongTensor)
    return X, y

# Membagi dataset menjadi 3 bagian: training, validation, dan test set.

def split_data(X, y, b_size):
    training, test_x, training_label, test_y = train_test_split(X,y, test_size=0.2, random_state=42)
    train_x, val_x, train_y, val_y = train_test_split(training, training_label, test_size=0.2, random_state=42)

    train = TensorDataset(train_x, train_y)
    val = TensorDataset(val_x, val_y)
    test = TensorDataset(test_x, test_y)

    train_loader = DataLoader(train, batch_size=b_size)
    val_loader = DataLoader(val, batch_size=b_size)
    test_loader = DataLoader(test, batch_size=b_size)

    print(f'train size: {len(train_loader.dataset)}, val size: {len(val_loader.dataset)}, test size: {len(test_loader.dataset)}')
    print(f'train num bacthes: {len(train_loader)}, val : {len(val_loader)}, test : {len(test_loader)}')
    return train_loader, val_loader, test_loader

## Membangun arsitektur model LSTM

In [ ]:
class Lstmiris(nn.Module):
    def __init__(self,input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()

        #hidden dim
        self.hidden_dim = hidden_dim

        #number of hidden layers
        self.layer_dim = layer_dim

        #LSTM
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, dropout=0.2, batch_first=True)

        #readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # time steps
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :])
        out = self.dropout(out)
        # out.size() --> 100, 10
        #print(out.size())
        return out

## Menampilkan Confusion Matrix

## Tahap Training

## Tahap Validasi dan Uji

## Main Program

In [11]:
# memuat data
path = '../machinel/data/Iris.csv'
X, y = get_data(path)


#split data
batch_size = 5
train, val, test = split_data(X, y, batch_size)
# memulai training
#training
#validation

#done

train size: 96, val size: 24, test size: 30
train num bacthes: 20, val : 5, test : 6


## Menampilkan plot untuk accuracy & loss dari model

## Melakukan pengujian dengan test set